# Usage of FlaUI for Python

FlaUI is a .NET library which helps with automated UI testing of Windows applications (Win32, WinForms, WPF, Store Apps, ...). It is based on native UI Automation libraries from Microsoft and therefore kind of a wrapper around them. FlaUI wraps almost everything from the UI Automation libraries but also provides the native objects in case someone has a special need which is not covered (yet) by FlaUI. Some ideas for things not yet implemented are at the end of this page.

FlaUI.UIA3 is the .NET 4.5 version of FlaUI for the UI Automation library which is also used by Microsoft for CodedUI.

This is just a quick example of how it looks like to automate a WPF Application/WinForms application with FlaUI. It is not a full documentation.


## Setting up CLR bridge

We need to setup the CLR bridge to be able to use FlaUI from Python. This is done by importing the clr module and adding the path to the FlaUI dlls to the sys.path. The path is relative to the current file.

The required FlaUI binaries are packaged into the disted Python package. DO NOTE that you need to run this function only once before you can call any further classes/methods from FlaUI.


In [1]:
from flaui.lib.pythonnet_bridge import setup_pythonnet_bridge

setup_pythonnet_bridge()

2023-10-01 18:29:03.519 | INFO     | flaui.lib.pythonnet_bridge:setup_pythonnet_bridge:20 - Added FlaUI.Core.dll DLL to Python.NET bridge
2023-10-01 18:29:03.535 | INFO     | flaui.lib.pythonnet_bridge:setup_pythonnet_bridge:20 - Added FlaUI.UIA2.dll DLL to Python.NET bridge
2023-10-01 18:29:03.554 | INFO     | flaui.lib.pythonnet_bridge:setup_pythonnet_bridge:20 - Added FlaUI.UIA3.dll DLL to Python.NET bridge
2023-10-01 18:29:03.554 | INFO     | flaui.lib.pythonnet_bridge:setup_pythonnet_bridge:20 - Added Interop.UIAutomationClient.dll DLL to Python.NET bridge
2023-10-01 18:29:03.554 | INFO     | flaui.lib.pythonnet_bridge:setup_pythonnet_bridge:24 - Python.NET bridge setup complete


## Using FlaUI from Python

Now we can use FlaUI from Python. First we need to import the FlaUI.UIA3 namespace. This namespace contains all the classes we need to automate a WPF application.

FlaUI has 2 valid namespaces that can be used for automation. FlaUI.UIA2 is the old namespace which uses the UI Automation library from .NET 3.5. This library is deprecated and should not be used anymore. The new library is FlaUI.UIA3 which uses the UI Automation library from .NET 4.5. This library is also used by Microsoft for CodedUI.

FlaUI.UIA2 is still applicable on certain frameworks like WinForms or WPF. For WinForms, FlaUI.UIA2 is the only applicable library.

### Upcoming changes for planned v1.0.0 release

When I dist v1.0.0 there would be major changes in the way you import the FlaUI.UIA3 namespace. The new way would be:

```python
from flaui.uia3 import UIA3Automation # Python
```

Comparing this to the old way:

```csharp
using FlaUI.UIA3; // C#
```

```python
from FlaUI.UIA3 import UIA3Automation # Python
```

If you compare the old method, ideally examples in this document import the C# namespace and then use the classes from the namespace. The new method would import the Python namespace and then use the classes from the namespace. This is a major change which should ease the adoption of FlaUI for Python users.


## Automating a WPF Application

Now we can start automating a WPF application. First we need to get an automation object for the application. This is done by creating an Application object and attaching to the process. The process can be started by the Application object itself or by using the System.Diagnostics.Process class.

```python
from FlaUI.UIA3 import UIA3Automation
```


In [2]:
from FlaUI.UIA3 import UIA3Automation
from FlaUI.Core import Application

In [3]:
application = Application.Launch(r"C:\Users\\Amruth.Vithala\\Projects\flaui-uiautomation-wrapper\test_applications\\WPFApplication\\WpfApplication.exe")

You could attach to an existing application by passing in the Process name as in `notepad.exe` or Process ID. If you want to start a new application, you can pass in the path to the executable and the arguments to the Application constructor.

```python
from FlaUI.UIA3 import UIA3Automation

automation = UIA3Automation()
app = automation.launch('C:\Windows\System32\notepad.exe')
```

Now we can start automating the application. The first thing we want to do is to get the main window of the application. This is done by calling the GetMainWindow method on the Application object. This method returns an AutomationElement which represents the main window of the application.

```python
from FlaUI.UIA3 import UIA3Automation

automation = UIA3Automation()
app = automation.launch('C:\Windows\System32\notepad.exe')
mainWindow = app.getMainWindow(automation)
```

Here's an example of attaching ot an existing application and getting the main window:

```python
from FlaUI.UIA3 import UIA3Automation

automation = UIA3Automation()
app = automation.attach('notepad')
mainWindow = app.getMainWindow()
```

If you are working with multiple applications and want to use this library, create an instance of the application by using `application.launch` or `application.attach` and then use the `application.getMainWindow` method to get the main window of the application. This is the starting point for all automation.


In [4]:

uia3_automation = UIA3Automation()

main_window = application.GetMainWindow(uia3_automation)
print(f"Window title is - {main_window.Title}")


Window title is - FlaUI WPF Test App


### Finding elements with XPath

FlaUI supports finding elements with XPath. This is a very powerful way to find elements. The XPath is evaluated by the UI Automation library and therefore the XPath is not limited to the properties of the elements but can also use the properties of the elements parents. This is very useful for example to find a button with a certain text. The text of a button is not a property of the button itself but of the text element inside the button. With XPath, this is no problem.


In [5]:
x_path_element = main_window.FindFirstByXPath("/Tab/TabItem[@Name='Simple Controls']")
print(f"Element name is - {x_path_element.Name}")

Element name is - Simple Controls


## Finding elements with conditions

FlaUI supports finding elements with conditions. This is a very powerful way to find elements. The conditions are evaluated by the UI Automation library and therefore the conditions are not limited to the properties of the elements but can also use the properties of the elements parents. This is very useful for example to find a button with a certain text. The text of a button is not a property of the button itself but of the text element inside the button. With conditions, this is no problem.

To use conditions, we need to invoke a condition factory from the automation object. The condition factory is used to create conditions. The conditions can then be used to find elements.


You can use conditions to find elements by using the `findFirst` method on the `AutomationElement` class. This method takes a condition as parameter and returns the first element which matches the condition. If no element matches the condition, null is returned.

```python
condition_factory = uia3_automation.ConditionFactory

name_condition = condition_factory.ByName("Simple Controls")
element.FindFirstChild(name_condition)
```


In [6]:
from FlaUI.Core.Definitions import ControlType

condition_factory = uia3_automation.ConditionFactory
element_by_condition = main_window.FindFirstChild(condition_factory.ByControlType(ControlType.Tab)).FindFirstChild(condition_factory.ByName("Simple Controls"))
print(f"Element name is - {element_by_condition.Name}")

Element name is - Simple Controls


## Usage for WinForms applications or any applications using UIA2

FlaUI.UIA2 is the old namespace which uses the UI Automation library from .NET 3.5. This library is deprecated and should not be used anymore. The new library is FlaUI.UIA3 which uses the UI Automation library from .NET 4.5. This library is also used by Microsoft for CodedUI.

FlaUI.UIA2 is still applicable on certain frameworks like WinForms or WPF. For WinForms, FlaUI.UIA2 is the only applicable library.

Usage of this library is almost the same as for FlaUI.UIA3. The only difference is that you need to use the `UIA2Automation` class instead of the `UIA3Automation` class.

```python
from FlaUI.UIA2 import UIA2Automation

automation = UIA2Automation()
app = automation.launch('C:\Windows\System32\notepad.exe')
mainWindow = app.getMainWindow(automation)
```


## Close the application

To close the application, you can use the `Close` method on the `Application` object.

```python
application.close()
```


In [7]:
# Close the application
application.Close()

True

## End-Notes

This is just a quick example of how it looks like to automate a WPF Application/WinForms application with FlaUI. It is not a full documentation.

If you intend to use this library alongside existing implementations of other test frameworks like [Robot-FlaUI](https://github.com/GDATASoftwareAG/robotframework-flaui), you should create Robot keywords that wrap the functionality of this library. This way you can use the keywords from Robot-FlaUI and the keywords from this library in the same test suite.

### Example with Robot-FlaUI

#### Robot file

```robot
*** Settings ***
Library    FlaUILibrary

*** Test Cases ***
Test
    Launch Application    C:\\Windows\\System32\\notepad.exe
    ${mainWindow}=    Get Main Window
    ${button}=    Find Element    ByXPath    //Button[@Name='OK']
    Click    ${button}
    Close Application
```

#### Python file with FlaUI

```python
from FlaUI.UIA3 import UIA3Automation

automation = UIA3Automation()
app = automation.launch('C:\Windows\System32\notepad.exe')
mainWindow = app.getMainWindow(automation)
button = mainWindow.findFirstByXPath('//Button[@Name="OK"]')
button.click()
app.close()
```
